In [37]:
import numpy
from keras import Sequential
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Dense
import os
import pandas as pd
import numpy as np
from time import time as t
from sklearn.metrics import classification_report
import h5py
from keras import optimizers
from keras import callbacks
import time
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import pickle

In [38]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ParameterGrid


In [39]:
from keras import backend as K

def tp_m(y_true, y_pred):
    #true_positives = K.sum(y_true * K.round(y_pred))
    return y_pred
    
def fp_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return predicted_positives-true_positives

def tn_m(y_true, y_pred):
    print('y_true - 1 '+str((y_true-1)))
    false_negatives = K.sum(K.round(K.clip((y_true-1) * (y_pred-1), 0, 1)))
    return false_negatives
    
def fn_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return possible_positives-true_positives

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [41]:
numpy.random.seed(0)

In [42]:
n_splits=4
train_size=0.8
test_size=0.2

In [43]:
class EEGAlcoDatasetBalanced():
    """EEG Train dataset."""

    def __init__(self):
        """
        Args:
            none.
        """
        h5f = h5py.File('alco_scalars_balanced_X_train.h5','r')
        self.spikes_seizure_eeg_train = h5f['dataset_alco_scalars_balanced_X_train'][:]
        #self.spikes_seizure_eeg_train=np.swapaxes(self.spikes_seizure_eeg_train,1,2)
        h5f.close()
        
        h5f = h5py.File('alco_scalars_balanced_X_test.h5','r')
        self.spikes_seizure_eeg_test = h5f['dataset_alco_scalars_balanced_X_test'][:]
        #self.spikes_seizure_eeg_test=np.swapaxes(self.spikes_seizure_eeg_test,1,2)

        #print(str(np.sum(self.labels_seizure_eeg))+'/'+str(len(self.labels_seizure_eeg)))
        h5f.close()
        
        h5f = h5py.File('alco_balanced_y_train.h5','r')
        self.labels_seizure_eeg_train = h5f['dataset_alco_balanced_y_train'][:]
        #self.spikes_seizure_eeg=np.swapaxes(self.spikes_seizure_eeg,1,2)
        h5f.close()
        
        h5f = h5py.File('alco_balanced_y_test.h5','r')
        self.labels_seizure_eeg_test = h5f['dataset_alco_balanced_y_test'][:]
        #self.spikes_seizure_eeg=np.swapaxes(self.spikes_seizure_eeg,1,2)
        h5f.close()

    def get_data(self):
        #all folds
        dataArray = list()

        trainLabels=self.labels_seizure_eeg_train
        trainValues=self.spikes_seizure_eeg_train  
        testLabels=self.labels_seizure_eeg_test
        testValues=self.spikes_seizure_eeg_test

        shuffle = np.random.RandomState(seed=0).permutation(len(trainValues))
        trainValues = trainValues[shuffle]
        trainLabels = trainLabels[shuffle]

        currentSplit = {'X_train': (trainValues), 'X_test': (testValues), 
                        'y_train': (trainLabels), 'y_test': (testLabels)}
        dataArray.append(currentSplit)
        return dataArray

    
    def __len__(self):
        return len(self.spikes_seizure_eeg_train)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        eeg = torch.tensor(self.spikes_seizure_eeg_train[idx])
        print('eeg size (in getitem): '+str(eeg.size()))
        label = self.labels_seizure_eeg_train[idx]
            
        sample = {'eeg': eeg, 'label': label}
        return sample

In [44]:
alcoDataset = EEGAlcoDatasetBalanced()
dataArray = alcoDataset.get_data()

In [45]:
starting_point = 0

In [46]:
data = 'alco_lstm'

In [47]:
class TimeHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()
        print('epoch time start: '+str(self.epoch_time_start))

    def on_epoch_end(self, batch, logs={}):
        end_time = time.time() - self.epoch_time_start
        self.times.append(end_time)
        print('epoch time measured: '+str(end_time))



In [48]:
n_epochs=250

In [49]:
resultsFilename = str(data)+"_results_data_FINAL.txt"

In [50]:
if os.path.isfile(resultsFilename):
    print ("Results file exists")
else:
    print ("Results file doesn't exist, creating new file...")

overallPrecisionList = list()
overallRecallList = list()
overallAccuracyList = list()
overallF1List = list()
overallTPList = list()
overallTNList = list()
overallFPList = list()
overallFNList = list()
overallConvergenceEpochList = list()
overallTrainingTimeList = list()
overallMeanEpochTimeList = list()
with open(resultsFilename, "a") as text_file:
    print(f"Test Results :\n\n", file=text_file)
for fold in np.arange(1):
    time_callback = TimeHistory()
    early_stopping_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=250)
    best_checkpoint_callback = ModelCheckpoint('optimal_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
    
    currentFold = fold
    # Train the network.
    print("Begin training for fold " + str(currentFold) + "\n")
    start = t()
    # create the model
   
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, 
                   input_shape=(64,255)))
    #model.add(Dropout(0.2))
    model.add(LSTM(128))
    #model.add(Dropout(0.2))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy',f1_m,precision_m, recall_m, tp_m, fp_m, tn_m, fn_m])
    print(model.summary())

    modelHistory=model.fit(dataArray[currentFold]['X_train'], dataArray[currentFold]['y_train'], epochs=n_epochs,
              batch_size=50, 
              validation_data=(dataArray[currentFold]['X_test'], dataArray[currentFold]['y_test']),
              callbacks=[time_callback, best_checkpoint_callback])

    stoppingEpoch=early_stopping_callback.stopped_epoch
    modelTimes = time_callback.times
    
    historyF1 = modelHistory.history['val_f1_m']
    historyPrecision = modelHistory.history['val_precision_m']
    historyRecall = modelHistory.history['val_recall_m']
    historyAccuracy = modelHistory.history['val_acc']
    historyTP = modelHistory.history['val_tp_m']
    historyFP = modelHistory.history['val_fp_m']
    historyTN = modelHistory.history['val_tn_m']
    historyFN = modelHistory.history['val_fn_m']
    
    convergenceEpochs=str(historyAccuracy.index(max(historyAccuracy)))
    totalTrainingTime=np.sum(np.array(modelTimes[int(convergenceEpochs)]).astype(np.float))
    print('type(totalTrainingTime): '+str(type(totalTrainingTime)))
    print('type(convergenceEpochs): '+str(type(convergenceEpochs)))
    meanTimePerEpoch=totalTrainingTime/int(convergenceEpochs)

    
    with open(resultsFilename, "a") as text_file:
        print(f"Training complete for fold {str(currentFold)}\n", file=text_file)
        print(f"Model Times: {str(modelTimes)}\n", file=text_file)
        print(f"Epoch Times: {str(modelTimes[0:int(convergenceEpochs)])}\n", file=text_file)
        print(f"Fold {str(currentFold)} Test Metrics:\n", file=text_file)
        print(f"Fold TP: {str(historyTP)}\nFold TN: {str(historyTN)}\nFold FP: {str(historyFP)}\nFold FN: {str(historyFN)}\n", file=text_file)
        print(f"Fold Accuracy: {str(historyAccuracy)}\nFold Precision: {str(historyPrecision)}\nFold Recall: {str(historyRecall)}\nFold F1: {str(historyF1)}\n", file=text_file)
        print(f"Accuracy Maxes at Epoch: {str(convergenceEpochs)}\n", file=text_file)
        print(f"Early stopping at epoch: {str(stoppingEpoch)}\n", file=text_file)
        print(f"Training Epochs until max accuracy: {str(convergenceEpochs)}\n", file=text_file)
        print(f"Total Training Time: {str(totalTrainingTime)}\n", file=text_file)
        print(f"Mean Time per Epoch: {str(meanTimePerEpoch)}\n", file=text_file)

    # load the saved model
    saved_model = load_model('optimal_model.h5', custom_objects={"f1_m": f1_m,
                                                              "precision_m": precision_m,
                                                              "recall_m": recall_m,
                                                            "tp_m":tp_m, 
                                                             "fp_m":fp_m,
                                                             "tn_m":tn_m,
                                                             "fn_m":fn_m
                                                            })
    
    
    loss, accuracy, f1_score, precision, recall, tp, fp, tn, fn  = saved_model.evaluate(dataArray[currentFold]['X_test'], dataArray[currentFold]['y_test'], 
                            verbose=1)
    
    predictions=(saved_model.predict(dataArray[currentFold]['X_test'])>0.5).reshape(-1)
    truelabels=dataArray[currentFold]['y_test']==1
    tp=np.sum(np.logical_and(predictions, truelabels))
    tn=np.sum(np.logical_and(np.invert(predictions), np.invert(truelabels)))
    fp=np.sum(np.logical_and(predictions, np.invert(truelabels)))
    fn=np.sum(np.logical_and(np.invert(predictions), (truelabels)))
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    f1_score = (2*precision*recall)/(precision+recall)
    print(f"TP: {str(tp)}\nTN: {str(tn)}\nFP: {str(fp)}\nFN: {str(fn)}\n")
    print(f"Accuracy: {str(accuracy)}\nPrecision: {str(precision)}\nRecall: {str(recall)}\nF1: {str(f1_score)}\n")

    with open(resultsFilename, "a") as text_file:
        print(f"Final Test Metrics for {str(currentFold)}:\n", file=text_file)
        print(f"TP: {str(tp)}\nTN: {str(tn)}\nFP: {str(fp)}\nFN: {str(fn)}\n", file=text_file)
        print(f"Accuracy: {str(accuracy)}\nPrecision: {str(precision)}\nRecall: {str(recall)}\nF1: {str(f1_score)}\n", file=text_file)

    overallPrecisionList.append(historyPrecision)
    overallRecallList.append(historyRecall)
    overallAccuracyList.append(historyAccuracy)
    overallF1List.append(historyF1)
    overallTPList.append(historyTP)
    overallTNList.append(historyTN)
    overallFPList.append(historyFP)
    overallFNList.append(historyFN)
    overallConvergenceEpochList.append(convergenceEpochs)
    overallTrainingTimeList.append(totalTrainingTime)
    overallMeanEpochTimeList.append(meanTimePerEpoch)
    
precisionMeanOverFolds=np.average(np.array(overallPrecisionList), axis=0)
recallMeanOverFolds=np.average(np.array(overallRecallList), axis=0)
accuracyMeanOverFolds=np.average(np.array(overallAccuracyList), axis=0)
F1MeanOverFolds=np.average(np.array(overallF1List), axis=0)
TPMeanOverFolds = np.average(np.array(overallTPList), axis=0)
TNMeanOverFolds = np.average(np.array(overallTNList), axis=0)
FPMeanOverFolds = np.average(np.array(overallFPList), axis=0)
FNMeanOverFolds = np.average(np.array(overallFNList), axis=0)
convergenceMeanOverFolds = np.mean(np.array(overallConvergenceEpochList).astype(np.float))
totalTrainingTimeMeanOverFolds = np.mean(np.array(overallTrainingTimeList).astype(np.float))
EpochTimeMeanOverFolds = np.mean(np.array(overallMeanEpochTimeList).astype(np.float))


maxAccuracyIndex=int(str(np.argmax(accuracyMeanOverFolds)))


with open(str(resultsFilename), "a") as text_file:
    print(f"Training complete for all folds.", file=text_file)
    print(f"Mean Test Metrics Over All Folds:\n", file=text_file)
    print(f"Final Accuracy: {str(accuracyMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final Precision: {str(precisionMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final Recall: {str(recallMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final F1: {str(F1MeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final TP: {str(TPMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final FP: {str(FPMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final TN: {str(TNMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Final FN: {str(FNMeanOverFolds[maxAccuracyIndex])}\n", file=text_file)
    print(f"Obs. Until Max Accuracy: {str(convergenceMeanOverFolds)}\n", file=text_file)
    print(
        f"TP: {str(list(TPMeanOverFolds))}\nTN: {str(list(TNMeanOverFolds))}\nFP: {str(list(FPMeanOverFolds))}\nFN: {str(list(FNMeanOverFolds))}\n",
        file=text_file)
    print(
        f"Accuracy: {str(list(accuracyMeanOverFolds))}\nPrecision: {str(list(precisionMeanOverFolds))}\nRecall: {str(list(recallMeanOverFolds))}\nF1: {str(list(F1MeanOverFolds))}\n",
        file=text_file)

Results file exists
Begin training for fold 0

y_true - 1 Tensor("metrics_4/tn_m/sub:0", shape=(?, ?), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 64, 128)           196608    
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_5 (Dense)              (None, 30)                3870      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 31        
Total params: 332,093
Trainable params: 332,093
Non-trainable params: 0
_________________________________________________________________
None
Train on 600 samples, validate on 600 samples
Epoch 1/250
epoch time start: 1578167156.1546612
600/600 [==============================] - 4s 7ms/

600/600 [==============================] - 3s 5ms/step - loss: 0.0318 - acc: 0.9900 - f1_m: 0.9896 - precision_m: 0.9859 - recall_m: 0.9936 - tp_m: 0.5040 - fp_m: 0.3333 - tn_m: 24.6667 - fn_m: 0.1667 - val_loss: 2.3185 - val_acc: 0.5100 - val_f1_m: 0.3328 - val_precision_m: 0.4910 - val_recall_m: 0.2711 - val_tp_m: 0.4330 - val_fp_m: 10.5000 - val_tn_m: 14.5000 - val_fn_m: 14.0000
epoch time measured: 2.9348976612091064

Epoch 00015: val_acc did not improve
Epoch 16/250
epoch time start: 1578167202.603872
600/600 [==============================] - 3s 5ms/step - loss: 0.0109 - acc: 0.9967 - f1_m: 0.9964 - precision_m: 0.9931 - recall_m: 1.0000 - tp_m: 0.5000 - fp_m: 0.1667 - tn_m: 24.8333 - fn_m: 0.0000e+00 - val_loss: 2.3736 - val_acc: 0.4950 - val_f1_m: 0.3633 - val_precision_m: 0.4879 - val_recall_m: 0.3106 - val_tp_m: 0.5171 - val_fp_m: 13.1667 - val_tn_m: 11.8333 - val_fn_m: 12.0833
epoch time measured: 2.9651894569396973

Epoch 00016: val_acc did not improve
Epoch 17/250
epoch ti

600/600 [==============================] - 3s 5ms/step - loss: 0.0374 - acc: 0.9850 - f1_m: 0.9846 - precision_m: 0.9807 - recall_m: 0.9901 - tp_m: 0.5030 - fp_m: 0.5000 - tn_m: 24.5000 - fn_m: 0.2500 - val_loss: 2.0260 - val_acc: 0.5333 - val_f1_m: 0.3559 - val_precision_m: 0.4976 - val_recall_m: 0.2906 - val_tp_m: 0.4623 - val_fp_m: 10.5833 - val_tn_m: 14.4167 - val_fn_m: 12.7500
epoch time measured: 2.872218370437622

Epoch 00031: val_acc did not improve
Epoch 32/250
epoch time start: 1578167249.6081023
600/600 [==============================] - 3s 5ms/step - loss: 0.0377 - acc: 0.9850 - f1_m: 0.9847 - precision_m: 0.9847 - recall_m: 0.9855 - tp_m: 0.5053 - fp_m: 0.4167 - tn_m: 24.5833 - fn_m: 0.3333 - val_loss: 2.0388 - val_acc: 0.5367 - val_f1_m: 0.3930 - val_precision_m: 0.5066 - val_recall_m: 0.3428 - val_tp_m: 0.5132 - val_fp_m: 11.9167 - val_tn_m: 13.0833 - val_fn_m: 11.2500
epoch time measured: 2.9994773864746094

Epoch 00032: val_acc did not improve
Epoch 33/250
epoch time s

600/600 [==============================] - 3s 5ms/step - loss: 0.0103 - acc: 0.9983 - f1_m: 0.9984 - precision_m: 0.9968 - recall_m: 1.0000 - tp_m: 0.5042 - fp_m: 0.0833 - tn_m: 24.9167 - fn_m: 0.0000e+00 - val_loss: 2.5615 - val_acc: 0.5050 - val_f1_m: 0.4009 - val_precision_m: 0.4959 - val_recall_m: 0.3706 - val_tp_m: 0.5835 - val_fp_m: 14.3333 - val_tn_m: 10.6667 - val_fn_m: 10.4167
epoch time measured: 2.9670157432556152

Epoch 00047: val_acc did not improve
Epoch 48/250
epoch time start: 1578167297.0083525
600/600 [==============================] - 3s 5ms/step - loss: 0.0107 - acc: 0.9967 - f1_m: 0.9965 - precision_m: 1.0000 - recall_m: 0.9931 - tp_m: 0.4957 - fp_m: 0.0000e+00 - tn_m: 25.0000 - fn_m: 0.1667 - val_loss: 2.6927 - val_acc: 0.4883 - val_f1_m: 0.4175 - val_precision_m: 0.4999 - val_recall_m: 0.4039 - val_tp_m: 0.6238 - val_fp_m: 16.0000 - val_tn_m: 9.0000 - val_fn_m: 9.5833
epoch time measured: 2.8934273719787598

Epoch 00048: val_acc did not improve
Epoch 49/250
epoch

600/600 [==============================] - 3s 5ms/step - loss: 0.0065 - acc: 0.9983 - f1_m: 0.9981 - precision_m: 1.0000 - recall_m: 0.9964 - tp_m: 0.5012 - fp_m: 0.0000e+00 - tn_m: 25.0000 - fn_m: 0.0833 - val_loss: 2.4506 - val_acc: 0.5200 - val_f1_m: 0.4235 - val_precision_m: 0.4965 - val_recall_m: 0.4006 - val_tp_m: 0.6203 - val_fp_m: 15.2500 - val_tn_m: 9.7500 - val_fn_m: 8.7500
epoch time measured: 2.94858455657959

Epoch 00063: val_acc did not improve
Epoch 64/250
epoch time start: 1578167344.4190779
600/600 [==============================] - 3s 5ms/step - loss: 0.0015 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 0.4999 - fp_m: 0.0000e+00 - tn_m: 25.0000 - fn_m: 0.0000e+00 - val_loss: 2.5854 - val_acc: 0.5067 - val_f1_m: 0.4094 - val_precision_m: 0.4946 - val_recall_m: 0.3817 - val_tp_m: 0.6094 - val_fp_m: 15.0833 - val_tn_m: 9.9167 - val_fn_m: 9.5833
epoch time measured: 2.940906524658203

Epoch 00064: val_acc did not improve
Epoch 65/250
epoch 

...

epoch time measured: 2.942241668701172

Epoch 00188: val_acc did not improve
Epoch 189/250
epoch time start: 1578167716.0633905
600/600 [==============================] - 3s 5ms/step - loss: 8.0514e-06 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 0.5000 - fp_m: 0.0000e+00 - tn_m: 25.0000 - fn_m: 0.0000e+00 - val_loss: 3.7330 - val_acc: 0.5367 - val_f1_m: 0.4233 - val_precision_m: 0.5006 - val_recall_m: 0.3994 - val_tp_m: 0.6044 - val_fp_m: 14.0833 - val_tn_m: 10.9167 - val_fn_m: 9.0833
epoch time measured: 2.960676908493042

Epoch 00189: val_acc did not improve
Epoch 190/250
epoch time start: 1578167719.0242326
600/600 [==============================] - 3s 5ms/step - loss: 7.9461e-06 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 0.5000 - fp_m: 0.0000e+00 - tn_m: 25.0000 - fn_m: 0.0000e+00 - val_loss: 3.7372 - val_acc: 0.5367 - val_f1_m: 0.4233 - val_precision_m: 0.5006 - val_recall_m: 0.3994 - val_tp_m: 0.6044 - val_fp_m:

epoch time measured: 2.983717918395996

Epoch 00219: val_acc did not improve
Epoch 220/250
epoch time start: 1578167808.39756
600/600 [==============================] - 3s 5ms/step - loss: 5.5449e-06 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 0.5000 - fp_m: 0.0000e+00 - tn_m: 25.0000 - fn_m: 0.0000e+00 - val_loss: 3.8436 - val_acc: 0.5367 - val_f1_m: 0.4233 - val_precision_m: 0.5006 - val_recall_m: 0.3994 - val_tp_m: 0.6033 - val_fp_m: 14.0833 - val_tn_m: 10.9167 - val_fn_m: 9.0833
epoch time measured: 3.013258934020996

Epoch 00220: val_acc did not improve
Epoch 221/250
epoch time start: 1578167811.4109633
600/600 [==============================] - 3s 5ms/step - loss: 5.4846e-06 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 0.5000 - fp_m: 0.0000e+00 - tn_m: 25.0000 - fn_m: 0.0000e+00 - val_loss: 3.8473 - val_acc: 0.5367 - val_f1_m: 0.4233 - val_precision_m: 0.5006 - val_recall_m: 0.3994 - val_tp_m: 0.6033 - val_fp_m: 1

epoch time measured: 2.9737071990966797

Epoch 00250: val_acc did not improve
type(totalTrainingTime): <class 'numpy.float64'>
type(convergenceEpochs): <class 'str'>


/usr/lib/python3/dist-packages/ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in double_scalars


y_true - 1 Tensor("metrics_5/tn_m/sub:0", shape=(?, ?), dtype=float32)
600/600 [==============================] - 1s 2ms/step
TP: 122
TN: 213
FP: 87
FN: 178

Accuracy: 0.5583333333333333
Precision: 0.583732057416268
Recall: 0.4066666666666667
F1: 0.47937131630648333

